In [105]:
import math
import numpy as np # Линейная алгебра и многомерные массивы
import pandas as pd # Работа с таблицами данных
import matplotlib.pyplot as plt # Графическая библиотека
from ipywidgets import widgets, interact # Интерактивные виджеты
import seaborn as sns
import statsmodels.api as sm
from google.colab import drive
import plotly.graph_objects as go
import plotly.express as px

In [130]:
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/GDP.csv'
data_quater = pd.read_csv(file_path)

file_path = '/content/drive/My Drive/GDP-2.csv'
data_year = pd.read_csv(file_path)

file_path = '/content/drive/My Drive/GDP-3.csv'
data_semian = pd.read_csv(file_path)

file_path = '/content/drive/MyDrive/TTLCONS.csv'
data_constructions = pd.read_csv(file_path)

data_quater.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,DATE;GDP
0,1947-01-01;243.164
1,1947-04-01;245.968
2,1947-07-01;249.585
3,1947-10-01;259.745
4,1948-01-01;265.742


Преобразуем представление данных, чтобы с ними было удобно работать

In [131]:
data_quater[['DATE', 'GDP']] = data_quater['DATE;GDP'].str.split(';', expand=True)
data_quater['GDP'] = pd.to_numeric(data_quater['GDP'])

data_quater.drop(columns=['DATE;GDP'], inplace=True)

Смерджим эти таблицы - нам далее они потребуются

In [132]:
merged_data = pd.merge(data_constructions, data_quater, on="DATE", how="inner")
merged_data

,DATE,TTLCONS,GDP
0,1993-01-01,458080.0,6729.459
1,1993-04-01,469425.0,6808.939
2,1993-07-01,483571.0,6882.098
3,1993-10-01,492823.0,7013.738
4,1994-01-01,513314.0,7115.652
...,...,...,...
119,2022-10-01,1830477.0,26408.405
120,2023-01-01,1882199.0,26813.601
121,2023-04-01,1907837.0,27063.012
122,2023-07-01,1969005.0,27610.128


Разложим при помощи фильтра Ходрика-Прескотта ВВП США на кварталам, по годам и  на циклическую и трендовую компоненты. В статье создателей фильтра рекомендуемое значение множителя Лагранжа для ежеквартальных данных сооставляет $λ = 1600^*$.


*Hodrick R. J., Prescott E. C. Postwar US business cycles: an empirical investigation (англ.) // Journal of Money, credit, and Banking. — 1997. — P. 1-16.*

In [136]:
cycle_q, trend_q = sm.tsa.filters.hpfilter(data_quater['GDP'], 1600)

In [137]:
data_quater['cycle'] = cycle_q
data_quater['trend'] = trend_q

data_quater.head()

,DATE,GDP,cycle,trend
0,1947-01-01,243.164,4.520901,238.643099
1,1947-04-01,245.968,2.232656,243.735344
2,1947-07-01,249.585,0.754586,248.830414
3,1947-10-01,259.745,5.812469,253.932531
4,1948-01-01,265.742,6.695613,259.046387


In [138]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_quater['DATE'], y=data_quater['cycle'], mode='lines', name='Cycle'))
fig.add_trace(go.Scatter(x=data_quater['DATE'], y=data_quater['trend'], mode='lines', name='Trend'))
fig.add_trace(go.Scatter(x=data_quater['DATE'], y=data_quater['GDP'], mode='lines', name='GDP'))

crossings = np.where(np.diff(np.sign(data_quater['trend'] - data_quater['GDP'])))[0]
crossing_dates = data_quater['DATE'].iloc[crossings]

fig.add_trace(go.Scatter(x=crossing_dates, y=data_quater['GDP'].iloc[crossings], mode='markers',
                         marker=dict(color='red', size=10), name='Crossings'))

fig.show()

Отдельно посмотрим, как вела себя циклическая составляющая за изучаемый период

In [139]:
fig_cyc = go.Figure()

fig_cyc.add_trace(go.Scatter(x=data_quater['DATE'], y=data_quater['cycle'], mode='lines', name='Cycle'))

fig_cyc.show()

Здесь довольно значительно выделяется момент структурного кризиса 2020 года

Наложим фильтр на годовые даные о ВВП США, рекомендуемый автворами параметр $λ = 100$

In [140]:
cycle_y, trend_y = sm.tsa.filters.hpfilter(data_year['GDP'], 100)
data_year['cycle'] = cycle_y
data_year['trend'] = trend_y

data_year

,DATE,GDP,cycle,trend
0,1947-01-01,249.61550,2.822578,246.792922
1,1948-01-01,274.46775,6.483643,267.984107
2,1949-01-01,272.47525,-16.728268,289.203518
3,1950-01-01,299.82725,-10.716966,310.544216
4,1951-01-01,346.91325,14.981268,331.931982
...,...,...,...,...
72,2019-01-01,21521.39500,-228.600021,21749.995021
73,2020-01-01,21322.94950,-1434.811622,22757.761122
74,2021-01-01,23594.03075,-225.796180,23819.826930
75,2022-01-01,25744.10825,827.267959,24916.840291


In [141]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_year['DATE'], y=data_year['cycle'], mode='lines', name='Cycle'))
fig.add_trace(go.Scatter(x=data_year['DATE'], y=data_year['trend'], mode='lines', name='Trend'))
fig.add_trace(go.Scatter(x=data_year['DATE'], y=data_year['GDP'], mode='lines', name='GDP'))

fig.show()

In [142]:
data_year.columns

Index(['DATE', 'GDP', 'cycle', 'trend'], dtype='object')

In [144]:
fig_combined = go.Figure()

fig_combined.add_trace(go.Scatter(x=data_year['DATE'], y=data_year['cycle'], mode='lines', name='Cycle (Yearly)'))
fig_combined.add_trace(go.Scatter(x=data_quater['DATE'], y=data_quater['cycle'], mode='lines', name='Cycle (Quarterly)'))

fig_combined.show()

В сравнении в квартальными данными годовые получаются более сглажеными. Важно отметить, что на графике хорошо выделаются периоды структурного спада в экономике - кризис 2008 года и кризис COVID-19 2020 года.

Идейно проседание циклической компоненты можно объяснить тем, что формально трендовая компонента представляет разность между реальными данными и трендовой составляющей. Момент шока в экономике сопровождается значительными отклонеием от тренда:
- так по полученному графику видно, что момент значительного роста экономики США набюдался в 2005-2007 годах. С точки зрения экономики, это объясняется  значительным ростом рынка недвижимости (подробнее о взимосявзи - ниже). Это пример позитивного шока
- негативный шок продемонтрирован в значительной степени в период 2008- 2010 года, а также в период корона-кризиса 2020-2021 года

Ниже я проверила существование корелляции между объемом рынка недвижимости и ВВП - ее значение свидетельствует о том, что значения эти показателей близки к зависимым. В целом, это довольно очевидная тенденция - стоимость строительства зданий входит в состав ВВП.

In [104]:
correlation = merged_data['TTLCONS'].corr(merged_data['GDP'])

fig = px.scatter(merged_data, x='TTLCONS', y='GDP', title=f'Correlation between TTLCONS and GDP: {correlation}',
                 labels={'TTLCONS': 'TTLCONS', 'GDP': 'GDP'}, trendline='ols')

fig.show()

На графике динамики объемов строительства в США, видно, что рост этой компоненты в 2005-2007 годах ковертировался в рост ВВП.

In [106]:
fig_constr = go.Figure()

fig_constr.add_trace(go.Scatter(x=data_constructions['DATE'], y=data_constructions['TTLCONS'], mode='lines', name='Cycle'))

fig_constr.show()

Следующей подзадачей является поиск оптимального значеия параметра $\lambda$ для полугодовых данных. В исходой статье такой информамции не было найдено, но сейчас предлагается подобрать значение параметра, решающее задачу минимизации целевого Лагранжиана фильтра.

Применим лобовой и не довольно тривиальный

In [152]:
def hp_filter(data, lamb):
    n = len(data)
    W = np.zeros((n, n))
    for i in range(1, n-1):
        W[i, i] = 1 + 2 * lamb
        W[i, i-1] = -lamb
        W[i, i+1] = -lamb
    W[0, 0] = 1 + lamb
    W[n-1, n-1] = 1 + lamb

    beta_hat = np.linalg.inv(W).dot(data)
    trend = beta_hat

    return trend


def simulated_annealing(data, initial_param, initial_temp, cooling_rate, num_iterations):
    current_param = initial_param
    current_temp = initial_temp

    for i in range(num_iterations):
        new_param = current_param + np.random.normal(0, 0.1)
        new_data = hp_filter(data, new_param)
        current_data = hp_filter(data, current_param)

        current_cost = np.sum((data - current_data)**2)
        new_cost = np.sum((data - new_data)**2)

        if new_cost < current_cost:
            current_param = new_param
        else:
            acceptance_probability = np.exp(-(new_cost - current_cost) / current_temp)
            if acceptance_probability > np.random.rand():
                current_param = new_param

        current_temp *= cooling_rate

    return current_param


initial_param = 1600
initial_temp = 2000
cooling_rate = 0.99
num_iterations = 1000

optimal_param = simulated_annealing(data_semian['GDP'].values, initial_param, initial_temp, cooling_rate, num_iterations)
print(f'Optimal parameter found: {optimal_param}')

Optimal parameter found: 1559.9838872626096


Оценка на оптимальное значение параметра $λ$ довольно соомительной получается - слишком велика значимость дисперсий между циклической составляющей и изменением темпа роста трендовой компоненты для данных с частотой изменений раз в полгода. Вероятно, следует выбрать более подходящий алгоритм, учитывающий особенности оптимизации данной нелинейной функции.

In [153]:
cycle_an, trend_an = sm.tsa.filters.hpfilter(data_semian['GDP'], optimal_param)

data_semian['cycle'] = cycle_an
data_semian['trend'] = trend_an

data_semian

,DATE,GDP,cycle,trend
0,1947-01-01,244.5660,3.070787,241.495213
1,1947-07-01,254.6650,2.565038,252.099962
2,1948-01-01,269.1545,6.447821,262.706679
3,1948-07-01,279.7810,6.462022,273.318978
4,1949-01-01,273.1925,-10.752104,283.944604
...,...,...,...,...
149,2021-07-01,24241.7880,141.381006,24100.406994
150,2022-01-01,25286.6945,633.974900,24652.719600
151,2022-07-01,26201.5220,991.561891,25209.960109
152,2023-01-01,26938.3065,1168.494043,25769.812457


In [154]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=data_semian['DATE'], y=data_semian['cycle'], mode='lines', name='Cycle'))
fig1.add_trace(go.Scatter(x=data_semian['DATE'], y=data_semian['trend'], mode='lines', name='Trend'))
fig1.add_trace(go.Scatter(x=data_semian['DATE'], y=data_semian['GDP'], mode='lines', name='GDP'))

fig1.show()

Изобразим циклические компоненты на графике:

In [156]:
fig_combined = go.Figure()

fig_combined.add_trace(go.Scatter(x=data_year['DATE'], y=data_year['cycle'], mode='lines', name='Cycle (Yearly)'))
fig_combined.add_trace(go.Scatter(x=data_quater['DATE'], y=data_quater['cycle'], mode='lines', name='Cycle (Quarterly)'))
fig_combined.add_trace(go.Scatter(x=data_semian['DATE'], y=data_semian['cycle'], mode='lines', name='Cycle (Semiannualy)'))

fig_combined.show()